In [2]:
import pandas as pd
import sqlite3

## Подключение через `sqlite3`

In [3]:
data = sqlite3.connect("../data/checking-logs.sqlite")

## Для `test` и `control` посчитать среднюю разницу `avg_diff` до и после первого посещения новостной ленты.

---

- Создается времення таблица `all_delts`

- `DATETIME(deadlines.deadlines, 'unixepoch') AS deadline`

    - Преобразует столбец `deadlines.deadlines` из таблицы `deadlines` в читаемый формат даты и времени

- `(deadlines.deadlines - unixepoch(first_commit_ts)) / 3600.0 AS delta_commit_deadline`

    - Вычисляет разницу во времени между дедлайном `deadlines.deadlines` и первым коммитом `first_commit_ts` в часах
    
    - Деление на 3600.0 для получения float

- `CASE WHEN first_view_ts <= first_commit_ts THEN 'after' ELSE 'before' END AS time`

    - Создает столбец `time`, который указывает, был ли первый просмотр страницы `first_view_ts` раньше или позже первого коммита `first_commit_ts`

- `FROM test JOIN deadlines ON deadlines.labs = test.labname`

    - Соединяет таблицу `test` с таблицей `deadlines` по условию - столбец `labs` в таблице `deadlines` совпадает со столбцом `labname` в таблице `test`. Это связывает каждую лабораторную работу с ее дедлайном

- `WHERE labname != 'project1'`

    - лючает строки, где `labname` в таблице `test` равен `'project1'`

- `time` -> `'befor'` / `'after'`

- `AVG(delta_commit_deadline) AS avg_diff`

    - Вычисляет среднее значение `delta_commit_deadline` - разница в часах между дедлайном и первым коммитом для каждой группы `time`

- `GROUP BY time`

    - Группирует результаты по столбцу `time`, создавая отдельные строки

In [4]:
sql_request = """
WITH all_delts AS (
    SELECT uid, 
           first_view_ts, 
           first_commit_ts, 
           DATETIME(deadlines.deadlines, 'unixepoch') AS deadline,
           (deadlines.deadlines - unixepoch(first_commit_ts)) / 3600.0 AS delta_commit_deadline,
           CASE
               WHEN first_view_ts <= first_commit_ts THEN 'after' 
               ELSE 'before' 
           END AS time
    FROM test
    JOIN deadlines ON deadlines.labs = test.labname
    WHERE labname != 'project1'
)
SELECT time, AVG(delta_commit_deadline) AS avg_diff 
FROM all_delts 
GROUP BY time
"""

test_res = pd.read_sql(sql_request, data)

test_res

,time,avg_diff
0,after,103.953446
1,before,61.156632


## В `control` вместо времени первого просмотра используется среднее значение из `test`

---

- `DATETIME(deadlines.deadlines, 'unixepoch') AS deadline`

    - Преобразует столбец `deadlines.deadlines` из таблицы `deadlines` в читаемый формат даты и времени

- `(deadlines - unixepoch(first_commit_ts)) / 3600 AS delta_commit_deadline`

    - Вычисляет разницу во времени между дедлайном `deadlines.deadlines` и первым коммитом `first_commit_ts` в часах
    
    - Деление на 3600 переводит секунды в часы

- `CASE WHEN first_view_ts <= first_commit_ts THEN 'after' ELSE 'before' END AS time`

    - Создает столбец `time`  указывает был ли первый просмотр раньше или позже либо одновременно с первым коммитом

        - `time = 'after'`
            
            - Если `first_view_ts <= first_commit_ts`
        
        - `time = 'before'`
            
            - Если `first_view_ts > first_commit_ts`

- `FROM control JOIN deadlines ON deadlines.labs = control.labname`

    - Соединяет таблицу `control` с таблицей `deadlines`, где столбец `labs` в `deadlines` совпадает со столбцом `labname` в `control`. Это связывает каждую лабораторную работу с ее дедлайном

- `WHERE labname != 'project1'`

    - Исключает строки, где `labname` в таблице `control` == `'project1'`

- `time` -> `'befor'` / `'after'`

- `AVG(delta_commit_deadline) AS avg_diff`

    - Вычисляет среднее значение `delta_commit_deadline` - разница в часах между дедлайном и первым коммитом для каждой группы `time`

- `GROUP BY time`

    - Группирует результаты по столбцу `time`, создавая отдельные строки

In [5]:
sql_request = """
WITH all_delts AS (
    SELECT uid,
           first_view_ts,
           first_commit_ts,
           DATETIME(deadlines.deadlines, 'unixepoch') AS deadline,
           (deadlines - unixepoch(first_commit_ts)) / 3600.0 AS delta_commit_deadline,
            CASE
                WHEN first_view_ts <= first_commit_ts THEN 'after'
                ELSE 'before'
            END AS time
    FROM control
    JOIN deadlines ON deadlines.labs = control.labname
    WHERE labname != 'project1'
)
SELECT time, AVG(delta_commit_deadline) AS avg_diff
FROM all_delts
GROUP BY time
"""

control_res = pd.read_sql(sql_request, data)

control_res

,time,avg_diff
0,before,107.576814


## Закрываем соединение с БД

In [6]:
data.close()